In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import gc
import sklearn


# load data
# ../input/h-and-m-personalized-fashion-recommendations/
original_df_customers = pd.read_csv('./customers.csv')
original_df_items = pd.read_csv('./articles.csv')
original_df_customers_items = pd.read_csv('./transactions_train.csv')

submission_file = pd.read_csv('./sample_submission.csv')

# CLEANING DATA CUSTOMERS

In [ ]:
original_df_customers = original_df_customers.drop(
    columns=['FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'postal_code'])


In [ ]:
original_df_customers['age'].fillna(
    (original_df_customers['age'].mean()), inplace=True)
original_df_customers['age'] = original_df_customers['age'].astype(np.int8)


In [ ]:
original_df_customers["age"].plot.hist(bins=50)


In [ ]:
bins = [20, 21, 55, 110]
labels = ['Teen', 'Adult', 'Senior']
original_df_customers['AgeGroup'] = pd.cut(
    original_df_customers['age'], bins=bins, labels=labels, right=False)


In [ ]:
original_df_customers


In [ ]:
original_df_customers = original_df_customers.drop(columns=['age'])


# CLEANING DATA ITEMS

In [ ]:
original_df_items.columns


In [ ]:
for i in range(len(original_df_items.columns)):
    print(original_df_items.columns[i] + " " + str(
        len(original_df_items[original_df_items.columns[i]].unique())))


In [ ]:
original_df_items['index_name'].unique()


In [ ]:
original_df_items.head(10)


In [ ]:
original_df_items = original_df_items[["article_id", "product_group_name",
                                       "perceived_colour_value_name", "perceived_colour_master_name", "index_name", "garment_group_name"]]


# CLEANING TRANSACTIONS

In [ ]:
original_df_customers_items = original_df_customers_items.drop(
    columns=['price', 'sales_channel_id'])


In [ ]:
# original_df_customers_items['day'] = original_df_customers_items['t_dat'].dt.day
original_df_customers_items['t_dat'] = pd.to_datetime(
    original_df_customers_items['t_dat'], format='%Y-%m-%d')
original_df_customers_items['day'] = original_df_customers_items['t_dat'].dt.day
original_df_customers_items['month'] = original_df_customers_items['t_dat'].dt.month
original_df_customers_items['year'] = original_df_customers_items['t_dat'].dt.year


In [ ]:
original_df_customers_items['season'] = pd.Categorical.from_codes(
    original_df_customers_items['t_dat'].dt.month % 12 // 3,
    categories=['winter', 'spring', 'summer', 'fall'],
    ordered=True
)

In [ ]:
original_df_customers_items


In [ ]:
original_df_customers_items = original_df_customers_items.drop(columns=['t_dat', 'month'])


In [ ]:
value_counts = original_df_customers_items.article_id.value_counts()


In [ ]:
#counts_items = original_df_items.merge(value_counts, left_on='article_id', right_on='article_id')
counts_items = pd.DataFrame(original_df_customers_items.article_id.value_counts().reset_index())
counts_items.columns = ['article_id', 'count']

# merge 'df' & 'brands_count'
original_df_customers_items = pd.merge(
    original_df_customers_items, counts_items, on='article_id')


# MERGE DATASETS

In [ ]:
df_merged_1 = original_df_customers_items.merge(original_df_customers, left_on='customer_id', right_on='customer_id')
del original_df_customers_items
del original_df_customers
gc.collect()


In [ ]:
df_merged_final = df_merged_1.merge(original_df_items, left_on='article_id', right_on='article_id')
del original_df_items
gc.collect()


In [ ]:
df_merged_final.columns


In [ ]:
df_merged_final = df_merged_final[['customer_id', 'AgeGroup', 'article_id', 'product_group_name', 'perceived_colour_value_name',
                                   'perceived_colour_master_name', 'index_name', 'garment_group_name', 'count', 'day', 'year', 'season']]


In [ ]:
df_merged_final


In [ ]:
article_ids = np.unique(df_merged_final.article_id.values)
customer_ids = np.unique(df_merged_final.customer_id.values)

num_articles = len(article_ids)
num_customers = len(customer_ids)

# Create dictionaries with mapping keys
articles_id_to_idx = dict(zip(article_ids, range(num_articles)))
customers_id_to_idx = dict(zip(customer_ids, range(num_customers)))

articles_idx_to_id = dict(zip(range(num_articles), article_ids))
customers_idx_to_id = dict(zip(range(num_customers), customer_ids))


In [ ]:
articles_id_to_idx

In [ ]:
def top_bought_articles(df, n):
    return df.value_counts().head(n)


In [ ]:
top_12 = top_bought_articles(df_merged_final['article_id'], 12)

In [ ]:
top_12 = top_12.to_numpy()

In [ ]:
# Assign the customer and article indices to the transactions and drop the ids
df_merged_final['customer_index'] = df_merged_final.customer_id.map(
    customers_id_to_idx)
df_merged_final['article_index'] = df_merged_final.article_id.map(
    articles_id_to_idx)

df_merged_final.drop(['customer_id', 'article_id'], inplace=True, axis=1)


In [ ]:
df_merged_final = df_merged_final[[
    'customer_index', 'AgeGroup', 'article_index', 'product_group_name']]


In [ ]:
df_merged_final


In [ ]:
df_merged_final['sale'] = 1
data_aux = df_merged_final.copy()
data_aux['sale'] = 0  # -1

data_aux['customer_index'] = data_aux['customer_index'].sample(frac=1, ignore_index=True)
data_aux['article_index'] = data_aux['article_index'].sample(frac=1, ignore_index=True)


In [ ]:
df_merged_final = df_merged_final.merge(data_aux, how='outer')

# One Hot Encoding

In [ ]:
y = df_merged_final['sale']
X = df_merged_final.drop(['sale'], axis=1)

In [ ]:
columns = X.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder


def one_hot_encoding(data, categorical_cols, sparse=True, df=False):
    # instantiate labelencoder object
    le = LabelEncoder()

    # apply le on categorical feature columns
    data[categorical_cols] = data[categorical_cols].apply(
        lambda col: le.fit_transform(col))

    from sklearn.preprocessing import OneHotEncoder
    ohe = OneHotEncoder(sparse=True)

    # One-hot-encode the categorical columns.
    # Unfortunately outputs an array instead of dataframe.
    array_hot_encoded = ohe.fit_transform(data[categorical_cols])
    print(type(array_hot_encoded))

    if df == True:
        # Convert it to df
        data_hot_encoded = pd.DataFrame(array_hot_encoded, index=data.index)

        # Extract only the columns that didnt need to be encoded
        data_other_cols = data.drop(columns=categorical_cols)

        # Concatenate the two dataframes :
        data_out = pd.concat([data_hot_encoded, data_other_cols], axis=1)
        return le, ohe, data_out
    else:
        return le, ohe, array_hot_encoded


In [ ]:
le, ohe_func, array_hot_encoded = one_hot_encoding(
    X, columns, sparse=True, df=False)


In [ ]:
import scipy
scipy.sparse.save_npz('./sparse_matrix.npz', array_hot_encoded)
# sparse_matrix = scipy.sparse.load_npz('./sparse_matrix.npz')


## Lesson 4: Collaborative-based recommender systems: Factorization MODELS

Matrix factorization is a simple embedding model, which decomposes the user-item intraction matrix, $R \in R^{m\times n}$ matrix, where $m$ is the number of users and $n$ the number of items, into the product of two lower dimensionality rectangular matrice. The the goal of the factorization models is to learn:
* A user embedding (or user latent factor) $P \in R^{m\times k}$, where row $i$ is the embedding of user $i$.
* A item embedding (or item latent factor) $Q \in R^{n\times k}$, where row $j$ is the embedding of user $j$.

![alt factorization models](https://miro.medium.com/max/988/1*nIVWl2ROaxOY23hHajkTTg.png)

These methods became really popular due to the shown efficiency in the Netflix Prize Challenge. 

There are several variants of these methods:
* SVD 
* Vanilla Factorization Matrix (also knwon as Funk SVD)
* Vanilla Factorization Matrix with biases 
* SVD++
* ...

In these notewook we will the application of these method to the transactionLens dataset.

In [ ]:
from scipy.sparse import csr_matrix, dok_matrix


def to_dense(array):
    """
    Accepta una csr_matrix, dok_matrix o matrix i la converteix en una 
    np.array normal, densa.

    :param array: Array a convertir
    :return: np.array densa, sense cap dimensió de tamany 1
    """
    try:
        array = array.todense()
    except:
        pass

    return np.array(array).squeeze()


In [ ]:
def evaluate(predict_f, data_train, data_test):
    """ RMSE-based predictive performance evaluation with pandas. """
    ids_to_estimate = zip(data_test.customer_id, data_test.article_id)
    list_users = set(data_train.customer_id)
    estimated = np.array(
        [predict_f(u, i) if u in list_users else 0.5 for (u, i) in ids_to_estimate])
    real = data_test['sale'].values
    return compute_rmse(estimated, real)


def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))


# Divide the data in two sets: training and test
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.loc[sampled_ids, 'for_testing'] = True
    return df


### The Vanilla Matrix Factorization Model 
* Also know as **Funk SVD**
* * Despite its name, in Funk SVD, no singular value decomposition is applied.
* * https://sifter.org/simon/journal/20061211.html

A straightforward matrix factorization model maps both users and items to a joint latent factor space of dimensionality D. User-item interaction are modeled as inner products in that space
$$R = UV$$

Each item j is associated with a vector $v_j$ from $V$, and each user $i$ is associated with a vecor $u_i$ from $U$.
The resulting dot product $u_i\cdot v_j$ captures the interaction between the user $i$ and item $j$:
$$ \hat{r} = u_i\cdot v_j$$

The goal of the matrix factorization consist on finding the mapping of each item and user to factors $u_i$ and $v_j$. To do so, the minimization the of squarred error function is performed:
$$ \sum(R_{ui} - u_i\cdot v_j)^2$$

This factorization can be learnt using **only those known ratings**. We do not need to infer missing values.

![alt Amazon](https://miro.medium.com/max/4800/1*b4M7o7W8bfRRxdMxtFoVBQ.png)

In [ ]:
from scipy import sparse
import matplotlib.pyplot as plt
from numba import njit, jit


class RecSys_mf():
    def __init__(self, article_ids, customer_ids, num_components=10):
        """ Constructor """
        self.num_components = num_components

        self.article_ids = article_ids
        self.customer_ids = customer_ids

        self.num_articles = len(self.article_ids)
        self.num_customers = len(self.customer_ids)

        # Create dictionaries with mapping keys
        self.article_id2index = dict(zip(article_ids, range(num_articles)))
        self.customer_id2index = dict(zip(customer_ids, range(num_customers)))
        self.article_index2id = dict(zip(range(num_articles), article_ids))
        self.customer_index2id = dict(zip(range(num_customers), customer_ids))


class RecSys_vanilla_mf(RecSys_mf):

    def __init__(self, matrix, sales, article_ids, customer_ids, top_12, reg_w=0.01, reg_v=0.01, num_components=10, seed=42):
        """ Constructor """
        super().__init__(article_ids, customer_ids, num_components)
        self.matrix = matrix
        self.y = sales.to_numpy(dtype=np.int8)
        self.top_12 = top_12
        self.seed = seed   # Random seed

        self.reg_w = reg_w
        self.reg_v = reg_v

    def fit(self, n_epochs=10, learning_rate=0.001, lmbda=0.1, verbose=True):
        """ We decompose the R matrix into to submatrices using the training data """
        self.verbose = verbose
        self.learning_rate = learning_rate
        self.lmbda = lmbda

        self.n_samples, self.n_features = self.matrix.shape
        # Initialize weights
        self.w0 = np.random.normal(scale=1, size=1)
        self.w = np.random.normal(
            scale=1/self.n_features, size=self.n_features)
        self.v = np.random.normal(
            scale=1./self.n_features, size=(self.num_components, self.n_features))

        self.train_rmse = []
        
        print(type(self.y))

        for epoch in tqdm(range(n_epochs)):
            print('Epoch: {}'.format(epoch))

            self.training_indices = np.arange(self.n_samples)

            # self.__shuffle_sales__()
            # shuffle training samples
            np.random.shuffle(self.training_indices)
            __sdg__(self.matrix.data, self.matrix.indptr, self.matrix.indices, self.y, self.n_samples, self.n_features,
                    self.w0, self.w, self.v, self.num_components, self.learning_rate, self.reg_w, self.reg_v)

            # self.train_rmse.append(evaluate(self.predict, self.matrix, self.matrix))

            # print('\tTrain rmse: %s' % self.train_rmse[-1])

        if(self.verbose):
            self.__plot_learning_curves__()

    def __shuffle_sales__(self):
        """ Shuffle negative samples """
        self.y = np.random.shuffle(self.y)

    def __plot_learning_curves__(self):
        plt.plot(self.train_rmse, '--o', label="train_error")
        plt.plot(self.test_rmse, '--o', label="test_error")
        plt.legend()
        plt.show()

    def predict(self, customers):
        """Similar to _predict_instance but vectorized for all samples"""
        
        results = pd.DataFrame(columns=['customer_id', 'predict'])

        for idx, customer in enumerate(tqdm(customers.customer_id.unique())):
            if customer not in self.customer_id2index.keys():
                # If the the customer is not in the trained ones return the default recommendation
                article_ids = self.top_12
            else:
                customer_idx = self.customer_id2index[customer]
                
                summed = np.zeros(self.num_components)
                summed_squared = np.zeros(self.num_components)

                # linear output w * x
                pred = self.w0
                for index in range(self.matrix.indptr[customer_idx], self.matrix.indptr[customer_idx + 1]):
                    feature = self.matrix.indices[index]
                    pred += self.w[feature] * self.matrix.data[index]

                # factor output
                for factor in range(self.num_components):
                    for index in range(self.matrix.indptr[customer_idx], self.matrix.indptr[customer_idx + 1]):
                        feature = self.indices[index]
                        term = self.v[factor, feature] * self.matrix.data[index]
                        summed[factor] += term
                        summed_squared[factor] += term * term

                    pred += 0.5 * (summed[factor] * summed[factor] - summed_squared[factor])
                
                recommended = pred.argsort()[-12:]
                
                article_ids = ' '.join([self.article_index2id[item_idx] for item_idx in recommended])
            
            results.loc[idx] = (customer, ''.join(article_ids))
            
        return results


def __sdg__(data, indptr, indices, y, n_samples, n_features, w0, w, v, n_factors, learning_rate, reg_w, reg_v):
    """
    Compute the loss of the current iteration and update
    gradients accordingly.
    """
    loss = 0.0
    for i in range(n_samples):
        pred, summed = _predict_instance(
            data, indptr, indices, w0, w, v, n_factors, i)

        # calculate loss and its gradient
        loss += _log_loss(pred, y[i])
        loss_gradient = -y[i] / (np.exp(y[i] * pred) + 1.0)

        # update bias/intercept term
        w0 -= learning_rate * loss_gradient

        # update weight
        for index in range(indptr[i], indptr[i + 1]):
            feature = indices[index]
            w[feature] -= learning_rate * \
                (loss_gradient * data[index] + 2 * reg_w * w[feature])

        # update factor
        for factor in range(n_factors):
            for index in range(indptr[i], indptr[i + 1]):
                feature = indices[index]
                term = summed[factor] - v[factor, feature] * data[index]
                v_gradient = loss_gradient * data[index] * term
                v[factor, feature] -= learning_rate * \
                    (v_gradient + 2 * reg_v * v[factor, feature])

    loss /= n_samples
    return loss


def _predict_instance(data, indptr, indices, w0, w, v, n_factors, i):
    """predicting a single instance"""
    summed = np.zeros(n_factors)
    summed_squared = np.zeros(n_factors)

    # linear output w * x
    pred = w0
    for index in range(indptr[i], indptr[i + 1]):
        feature = indices[index]
        pred += w[feature] * data[index]

    # factor output
    for factor in range(n_factors):
        for index in range(indptr[i], indptr[i + 1]):
            feature = indices[index]
            term = v[factor, feature] * data[index]
            summed[factor] += term
            summed_squared[factor] += term * term

        pred += 0.5 * (summed[factor] * summed[factor] -
                       summed_squared[factor])

    # summed is the independent term that can be re-used
    # during the gradient update stage
    return pred, summed


def _log_loss(pred, y):
    """
    negative log likelihood of the
    current prediction and label, y.
    """
    return np.log(np.exp(-pred * y) + 1.0)


In [ ]:
reco = RecSys_vanilla_mf(array_hot_encoded, y, article_ids, customer_ids,
                         top_12, reg_w=0.01, reg_v=0.01, num_components=10, seed=42)
reco.fit(n_epochs=20, learning_rate=0.01, lmbda=0.5, verbose=True)


In [ ]:
result = reco._predict(submission_file)

In [ ]:
df_aux = result.explode(column='prediction')
df_aux.prediction = '0' + df_aux.prediction
df_aux.head()

In [ ]:
def group(list_of_strings):
    return " ".join(list_of_strings)
    
sub = df_aux.groupby('customer_id')['prediction'].agg(lambda x: group(x)).reset_index()

sub.to_csv('submission.csv', index=False)